# LangGraph Fundamentals: State Graphs and Message Management

This notebook introduces **LangGraph**, a framework for building stateful, cyclic agent workflows. Unlike traditional linear pipelines, LangGraph allows conditional branching, loops, and persistent state across multiple agent interactions.

**Core Concepts:**
1. **State**: A typed object (Pydantic BaseModel or TypedDict) representing workflow context
2. **Nodes**: Python functions that transform state
3. **Edges**: Connections between nodes, defining execution flow
4. **Reducers**: Functions that merge new state with existing state (e.g., `add_messages`)

In [ ]:
# Import dependencies
from typing import Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
import random

In [ ]:
# Initialize Environment
load_dotenv(override=True)

## Understanding State and Reducers

In LangGraph, `Annotated` is used to specify a **reducer function** that determines how new state values are merged with existing state. The built-in `add_messages` reducer appends new messages to a conversation history.

In [ ]:
# Define State Schema
class State(BaseModel):
    messages: Annotated[list, add_messages]

## Example 1: Simple Non-LLM Graph

Demonstrating that LangGraph is framework-agnostic—nodes can be any Python function.

In [ ]:
# Test Data
nouns = ["Cabbages", "Unicorns", "Toasters", "Penguins", "Bananas"]
adjectives = ["outrageous", "smelly", "existential", "sparkly", "sarcastic"]

# Node Function
def random_phrase_node(old_state: State) -> State:
    reply = f"{random.choice(nouns)} are {random.choice(adjectives)}"
    messages = [{"role": "assistant", "content": reply}]
    return State(messages=messages)

# Build Graph
graph_builder = StateGraph(State)
graph_builder.add_node("phrase_generator", random_phrase_node)
graph_builder.add_edge(START, "phrase_generator")
graph_builder.add_edge("phrase_generator", END)

# Compile
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Test Execution
initial_state = State(messages=[{"role": "user", "content": "Hello"}])
result = graph.invoke(initial_state)
print(result["messages"][-1].content)

## Example 2: LLM-Powered Chatbot Graph

In [ ]:
# LLM Client
llm = ChatOpenAI(model="gpt-4o-mini")

# Chatbot Node
def chatbot_node(old_state: State) -> State:
    response = llm.invoke(old_state.messages)
    return State(messages=[response])

# Build Graph
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot_node)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

# Compile
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Interactive Interface
def chat(user_input: str, history):
    initial_state = State(messages=[{"role": "user", "content": user_input}])
    result = graph.invoke(initial_state)
    return result['messages'][-1].content

gr.ChatInterface(chat, type="messages").launch()